In [ ]:
!pip install git+https://github.com/mathLab/EZyRB
![ ! -d "data" ] \
&& mkdir data \
&& wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_coord.npy \
&& wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_mu.npy \
&& wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_snapshots.npy \
&& wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_triangles.npy

In [ ]:
import numpy as np
import matplotlib.tri as mtri
import matplotlib.pyplot as plt

from ezyrb import POD, RBF, Database
from ezyrb import ReducedOrderModel as ROM
import numpy.linalg as LA


# ## Offline phase
# 
# In the *offline* phase, we need some samples of the parametric high-fidelity model. In this case, we extract 8 snapshots from the numerical model implemented in **FEniCS**, and we import them and the related parameters.


snapshots = np.load('data/tut1_snapshots.npy')
param = np.load('data/tut1_mu.npy')

print(snapshots.shape, param.shape)

train_snapshots = snapshots[0:7,:]
train_param = param[0:7,:]
print(train_snapshots.shape, train_param.shape)

test_snapshots = snapshots[7:8,:]
test_param = param[7:8,:]
print(test_snapshots.shape, test_param.shape)

# Moreover, to visualize the solution (both the higher-order one and the reduced one), we import also the mesh information to be able to create the triangulation. We underline this additional step is related only to plotting purpose, and not mandatory for the reduced space generation.


tri = np.load('data/tut1_triangles.npy')
coord = np.load('data/tut1_coord.npy')
triang = mtri.Triangulation(coord[0],coord[1],tri)


# For the sake of clarity the snapshots are plotted.


fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(16, 6), sharey=True, sharex=True)
ax = ax.flatten()
for i in range(8):
    ax[i].triplot(triang, 'b-', lw=0.1)
    cm = ax[i].tripcolor(triang, snapshots[i])
    fig.colorbar(cm, ax=ax[i])
    ax[i].set_title('($\mu_0={:5.2f}, \mu_1={:5.2f})$'.format(*param[i]))

plt.show()
# First of all, we create a `Database` object from the parameters and the snapshots.

db = Database(train_param, train_snapshots)



# Then we need a reduction object. In this case we use the proper orthogonal decomposition so we create a `POD` object. We use here all the default parameters, but for the complete list of available arguments we refer to original documentation of [POD](https://mathlab.github.io/EZyRB/pod.html) class.

pod = POD('svd')


# Then we instantiate the `RBF` class for interpolating the solution manifold. Also in this case, [RBF](https://mathlab.github.io/EZyRB/rbf.html) documentation is the perfect starting point to explore such class.


rbf = RBF()


# Few lines of code and our reduced model is created!
# To complete everything, we create the `ReducedOrderModel` (aliased to `ROM` in this tutorial) object by passing the already created objects. For clarity, we puntualize that we need to pass the **instances** and not the classes. Simply changing such line (with different objects) allows to test different frameworks in a very modular way.
# The `fit()` function computes the reduced model, meaning that the original snapshots in the database are projected onto the POD space and the RBF interpolator is created.

rom = ROM(db, pod, rbf)
rom.fit();


# ## Online phase
# In the *online* phase we can query our model in order to predict the solution for a new parameter $\mu_\text{new}$ that is not in the training set. We just need to pass the new parameters as input of the `predict()` function.


new_mu = test_param
pred_sol = rom.predict(new_mu)


# We can so plot the predicted solution for a fixed parameter...

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6), sharey=True, sharex=True)
ax[0].triplot(triang, 'b-', lw=0.1)
cm = ax[0].tripcolor(triang, snapshots[7])
fig.colorbar(cm, ax=ax[0])
ax[0].set_title('FOM: ($\mu_0={:5.2f}, \mu_1={:5.2f})$'.format(*param[7]))
ax[1].triplot(triang, 'b-', lw=0.1)
cm = ax[1].tripcolor(triang, pred_sol)
fig.colorbar(cm, ax=ax[1])
ax[1].set_title('ROM: ()$\mu_0={:5.2f}, \mu_1={:5.2f})$'.format(*param[7]))

plt.show()

print("pred_sol shape:", pred_sol.shape)
print("test_snapshots shape:", pred_sol.shape)

pred_error = pred_sol-test_snapshots
pred_error=pred_error[0,:]



plt.figure(figsize=(7, 5))
plt.triplot(triang, 'b-', lw=0.1)
plt.tripcolor(triang, pred_error)
plt.colorbar();
plt.show()

print("Prediction error: ", LA.norm(pred_error))
print("Relative prediction error: ", LA.norm(pred_error)/LA.norm(test_snapshots))

plt.plot(train_param[:,0],train_param[:,1],'*')
plt.plot(test_param[:,0],test_param[:,1],'r*')
plt.title('Train vs test parameter location')
plt.grid()
plt.show()

